In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
from stockapp import models as stkmd
from dataapp import models as dtamd
from dataapp import tasks as dtatks
from dataapp import libs as dtalibs
from stockapp import tasks as stktks
from stockapp import libs as stklibs
import utility as uty
import itertools as itt
import multiprocessing as mp
from django.db import connection,connections
from django.db import reset_queries
import time 
import pandas as pd
import numpy as np
import inspect
import imp
import datetime
import utility.models as utmd
import stockapp.libs as stklib
from utility import codemanager as cdmng
import json
from django.contrib.auth.models import AnonymousUser

In [27]:
print stkmd.ComputeStatus_Stockdownload.objects.filter(Status='Success').count()
print stkmd.ComputeStatus_Stockdownload.objects.filter(Status='Fail').count()
print stkmd.ComputeStatus_Stockdownload.objects.filter(Status='ToDo').count()

7098
842
0


In [28]:
stkgrpids=list(stkmd.StockGroup.objects.get(GroupName='Sector: Utilities').Symbol.all().values_list('id',flat=True))

In [15]:
Todate=pd.datetime.today().date()
Fromdate=pd.datetime(2002,1,1).date()
stk=stkmd.Stockmeta.objects.get(Symbol='UTIAvg')
df=dtalibs.GetStockData([stk.id],Fromdate,Todate,'concat')

 Time for GetStockData =  0.00325393676758


In [16]:
stk.Lastdate,stk.Startdate,stk.LastPriceUpdate

(None, None, None)

In [10]:
dtamd.Stockprice.objects.filter(Symbol_id=stk.id)

<QuerySet []>

In [13]:
stkmd.Stockmeta.objects.filter(Derived=True).update(Lastdate=None,Startdate=None,LastPriceUpdate=None)

1

# Add Index

# Building Query

In [6]:
pd.read_json('http://ec2-54-183-21-11.us-west-1.compute.amazonaws.com:8080/?symbols=TSLA,MSFT,AAPL&from=2016-01-01&to=2017-01-01')

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [10]:
url='http://ec2-54-183-21-11.us-west-1.compute.amazonaws.com:8080/?symbols=TSLA,MSFT,AAPL&from=2016-01-01&to=2017-01-01'
import urllib2
response = urllib2.urlopen(url)
html = response.read()
D=json.loads(html)

In [23]:
print len(D['result'])
# pd.DataFrame(D['result'])
df1=pd.DataFrame(D['result'][0])
df2=pd.DataFrame(D['result'][1])
df3=pd.DataFrame(D['result'][2])


3


In [26]:
df3.head()

,quotes,symbol
0,"{u'volume': 67649400, u'macd': None, u'rsi14':...",AAPL
1,"{u'volume': 55791000, u'macd': None, u'rsi14':...",AAPL
2,"{u'volume': 68457400, u'macd': None, u'rsi14':...",AAPL
3,"{u'volume': 81094400, u'macd': None, u'rsi14':...",AAPL
4,"{u'volume': 70798000, u'macd': None, u'rsi14':...",AAPL
